# Customer churn prediction
This notebook uses a subset (128MB) of the full dataset available (12GB) for the convinence of manipulating data and build machine learning model locally. 

In [233]:
# import libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, count, udf, to_timestamp, min, max, avg, when, length
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline


In [234]:
# create a Spark session
spark = SparkSession \
    .builder \
    .appName("Sparkify") \
    .getOrCreate()

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

### 1. Load and overview of the data

In [235]:
# Load and read data as dataframe
path = "mini_sparkify_event_data.json"
df = spark.read.json(path)
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [255]:
num_rows = df.count()
num_cols = len(df.columns)
print('The data set contains {} rows and {} columns'.format(num_rows, num_cols))

The data set contains 286500 rows and 18 columns


In [237]:
# Show the fist row of the dataframe
df.show(n=1)

+--------------+---------+---------+------+-------------+--------+---------+-----+---------------+------+--------+-------------+---------+---------+------+-------------+--------------------+------+
|        artist|     auth|firstName|gender|itemInSession|lastName|   length|level|       location|method|    page| registration|sessionId|     song|status|           ts|           userAgent|userId|
+--------------+---------+---------+------+-------------+--------+---------+-----+---------------+------+--------+-------------+---------+---------+------+-------------+--------------------+------+
|Martha Tilston|Logged In|    Colin|     M|           50| Freeman|277.89016| paid|Bakersfield, CA|   PUT|NextSong|1538173362000|       29|Rockpools|   200|1538352117000|Mozilla/5.0 (Wind...|    30|
+--------------+---------+---------+------+-------------+--------+---------+-----+---------------+------+--------+-------------+---------+---------+------+-------------+--------------------+------+
only showi

In [238]:
# Show all the columns
df.columns

['artist',
 'auth',
 'firstName',
 'gender',
 'itemInSession',
 'lastName',
 'length',
 'level',
 'location',
 'method',
 'page',
 'registration',
 'sessionId',
 'song',
 'status',
 'ts',
 'userAgent',
 'userId']

### 2. Check missing values and clean data

In [239]:
def check_missing_values(df):
    """Check the number of missing values in each columns and calculate the missing rate
    
    Arguments:
        df {dataframe} -- spark dataframe
        
    Return: None    
    """
    print('Column name         null count   null rate\n')
    for c in df.columns:
        null_count = df.filter(df[c].isNull() | (df[c] == '') ).count() # Check whether each column has null value or empty string
        null_rate = null_count/df.count() 
        print('{:15} | {:10} | {:10}'.format(c, null_count, null_rate))

In [240]:
check_missing_values(df)

Column name         null count   null rate

artist          |      58392 | 0.20381151832460734
auth            |          0 |        0.0
firstName       |       8346 | 0.029130890052356022
gender          |       8346 | 0.029130890052356022
itemInSession   |          0 |        0.0
lastName        |       8346 | 0.029130890052356022
length          |      58392 | 0.20381151832460734
level           |          0 |        0.0
location        |       8346 | 0.029130890052356022
method          |          0 |        0.0
page            |          0 |        0.0
registration    |       8346 | 0.029130890052356022
sessionId       |          0 |        0.0
song            |      58392 | 0.20381151832460734
status          |          0 |        0.0
ts              |          0 |        0.0
userAgent       |       8346 | 0.029130890052356022
userId          |       8346 | 0.029130890052356022


It is shown above that columns such as 'firstName', 'gender', 'lastName', 'location', 'registration' and 'userAgent' all have the same numbers of missing values as 'userId', because these information are all 'userId' related. Since our purpose in this project is to analyze user behavior, it would be save to drop these rows with no user information for further analysis. 


In [242]:
# drop rows without user id
df_dropId = df.filter(df['userId'] != '')

In [253]:
df_dropId.show(5)

+----------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|          artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page| registration|sessionId|                song|status|           ts|           userAgent|userId|
+----------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|  Martha Tilston|Logged In|    Colin|     M|           50| Freeman|277.89016| paid|     Bakersfield, CA|   PUT|NextSong|1538173362000|       29|           Rockpools|   200|1538352117000|Mozilla/5.0 (Wind...|    30|
|Five Iron Frenzy|Logged In|    Micah|     M|           79|    Long|236.09424| free|Boston-Cambridge-...|   PUT|NextSong|1538331630000| 

Another case is that 3 song-related columns 'artist', 'length' and 'song' have the same number of missing values. Let's check their interaction with 'page' column to get some idea. 

In [259]:
df.groupby('page').agg({'artist':'count', 'length':'count', 'song':'count'}).show(25)

+--------------------+-----------+-------------+-------------+
|                page|count(song)|count(artist)|count(length)|
+--------------------+-----------+-------------+-------------+
|              Cancel|          0|            0|            0|
|    Submit Downgrade|          0|            0|            0|
|         Thumbs Down|          0|            0|            0|
|                Home|          0|            0|            0|
|           Downgrade|          0|            0|            0|
|         Roll Advert|          0|            0|            0|
|              Logout|          0|            0|            0|
|       Save Settings|          0|            0|            0|
|Cancellation Conf...|          0|            0|            0|
|               About|          0|            0|            0|
| Submit Registration|          0|            0|            0|
|            Settings|          0|            0|            0|
|               Login|          0|            0|       

The three song related columns exclusively appears on 'NextSong' page. We will explore them further in feature engineering section.

Now, with missing user ids removed, let's see how many users in total are registered. 

In [261]:
# Check number of unique users
df.select('userId').distinct().count()

226

In [260]:
# pick some columns to get the detail
df.select('auth','level','page','registration','length','sessionId','song','status','userId').show(10)

+---------+-----+---------------+-------------+---------+---------+--------------------+------+------+
|     auth|level|           page| registration|   length|sessionId|                song|status|userId|
+---------+-----+---------------+-------------+---------+---------+--------------------+------+------+
|Logged In| paid|       NextSong|1538173362000|277.89016|       29|           Rockpools|   200|    30|
|Logged In| free|       NextSong|1538331630000|236.09424|        8|              Canada|   200|     9|
|Logged In| paid|       NextSong|1538173362000| 282.8273|       29|   Time For Miracles|   200|    30|
|Logged In| free|       NextSong|1538331630000|262.71302|        8|Knocking On Forbi...|   200|     9|
|Logged In| paid|       NextSong|1538173362000|223.60771|       29|Harder Better Fas...|   200|    30|
|Logged In| free|       NextSong|1538331630000|208.29995|        8|      Don't Leave Me|   200|     9|
|Logged In| free|       NextSong|1538331630000|260.46649|        8|      

In [74]:
# count occurences in each  'auth' category
df.groupby('auth').count().show()

+----------+------+
|      auth| count|
+----------+------+
|Logged Out|  8249|
| Cancelled|    52|
|     Guest|    97|
| Logged In|278102|
+----------+------+



In [76]:
# Count occurrences in each page category
df.groupby('page').count().show(30)

+--------------------+------+
|                page| count|
+--------------------+------+
|              Cancel|    52|
|    Submit Downgrade|    63|
|         Thumbs Down|  2546|
|                Home| 14457|
|           Downgrade|  2055|
|         Roll Advert|  3933|
|              Logout|  3226|
|       Save Settings|   310|
|Cancellation Conf...|    52|
|               About|   924|
| Submit Registration|     5|
|            Settings|  1514|
|               Login|  3241|
|            Register|    18|
|     Add to Playlist|  6526|
|          Add Friend|  4277|
|            NextSong|228108|
|           Thumbs Up| 12551|
|                Help|  1726|
|             Upgrade|   499|
|               Error|   258|
|      Submit Upgrade|   159|
+--------------------+------+



In [75]:
# Count occurrences in each  level category
df.groupby('level').count().show(22)

+-----+------+
|level| count|
+-----+------+
| free| 58338|
| paid|228162|
+-----+------+



In [194]:
df.select('userId').count()

286500

# Exploratory Data Analysis

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

1. At the moment, the subscription ends and renewal doesn’t happen, or

2. At the moment of the cancelation.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.